##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX 및 Vertex 파이프라인을 사용한 Vertex AI 훈련 및 서비스 제공


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Google Cloud Vertex AI Workbench에서 실행</a></td>
</table></div>


이 노트북 기반 튜토리얼은 Vertex AI Training 서비스를 사용하여 ML 모델을 훈련하고 서비스를 위해 이를 Vertex AI에 게시하는 TFX 파이프라인을 만들고 실행합니다.

이 노트북은 [Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)에서 구축한 TFX 파이프라인을 기반으로 합니다. 해당 튜토리얼을 아직 읽지 않았다면 이 노트북을 계속 진행하기 전에 꼭 읽기 바랍니다.

AutoML을 사용하여 Vertex AI에서 모델을 훈련하거나 사용자 지정 훈련을 사용할 수 있습니다. 사용자 지정 훈련에서는 다양한 시스템 유형을 선택하여 훈련 작업을 강화하고, 분산 훈련을 활성화하고, 하이퍼 매개변수 조정을 사용하고, GPU로 가속화할 수 있습니다.

학습된 모델을 Vertex AI 모델에 배포하고 엔드포인트를 생성하여 예측 요청을 제공할 수도 있습니다.

이 튜토리얼에서는 사용자 지정 작업과 함께 Vertex AI 훈련을 사용하여 TFX 파이프라인에서 모델을 훈련합니다. Vertex AI를 사용하여 예측 요청을 제공하는 모델도 배포합니다.


이 노트북은 [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) 또는 [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks)에서 실행하기 위한 것입니다. 이 중 하나도 사용하지 않는 경우, 위의 "Google Colab에서 실행하기" 버튼을 클릭하면 됩니다.

## 설정

[Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)을 마쳤다면 실제 작동하는 GCP 프로젝트와 GCS 버킷이 있을 것이며 이것이 이 튜토리얼에 필요한 전부입니다. 예비 튜토리얼을 아직 보지 않았다면 먼저 읽어보세요.

### Python 패키지 설치하기

TFX 및 KFP를 포함한 필수 Python 패키지를 설치하여 ML 파이프라인을 작성하고 Vertex 파이프라인에 작업을 제출합니다.

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### shapely 설치 제거하기

TODO(b/263441833) ImportError를 피하는 임시 솔루션입니다. 다른 추가 종속성을 제거하는 대신 최신 버전의 Bigquery를 지원하여 처리하는 것이 이상적입니다.

In [ ]:
!pip uninstall shapely -y

#### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 렁타임 다시 시작..." 메뉴를 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

Colab에 있지 않다면 다음 셀을 사용하여 런타임을 다시 시작할 수 있습니다.

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### 이 노트북을 사용하려면 Google에 로그인하세요

Colab에서 이 노트북을 실행하는 경우 사용자 계정으로 인증하세요.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**AI Platform Notebooks를 사용 중인 경우** 이어지는 섹션을 실행하기 전에 다음을 실행하여 Google Cloud에 인증합니다.

```sh
gcloud auth login
```

**터미널 창**(메뉴에서 **파일** &gt; **새로 만들기**를 통해 열 수 있음)에서 실행하면 됩니다. 이 작업은 노트북 인스턴스당 한 번만 수행하면 됩니다.

패키지 버전을 확인합니다.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### 변수 설정하기

아래에서 파이프라인을 사용자 지정하는 데 사용되는 몇 가지 변수를 설정합니다. 다음 정보가 필요합니다.

- GCP 프로젝트 ID. [프로젝트 ID 식별](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects)을 참조하세요.
- 파이프라인을 실행할 GCP 영역. Vertex 파이프라인을 사용할 수 있는 영역에 대한 자세한 내용은 [Vertex AI 위치 가이드](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)를 참조하세요.
- 파이프라인 출력을 저장하기 위한 Google Cloud Storage 버킷

**실행하기 전에 아래 셀에 필요한 값을 입력하세요**.


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

프로젝트를 사용하도록 `gcloud`를 설정합니다.

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-training'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Name of Vertex AI Endpoint.
ENDPOINT_NAME = 'prediction-' + PIPELINE_NAME

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### 예제 데이터 준비하기

[단순 TFX 파이프라인 튜토리얼](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)과 동일한 [Palmer Penguins 데이터세트](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)를 사용할 것입니다.

이 데이터세트에는 이미 [0,1] 범위를 갖도록 정규화된 4개의 숫자 요소가 있습니다. 펭귄의 `species`를 예측하는 분류 모델을 구축할 것입니다.

데이터세트의 자체 복사본을 만들어야 합니다. TFX ExampleGen은 디렉터리에서 입력을 읽기 때문에 디렉터리를 만들고 GCS에서 여기에 데이터세트를 복사해야 합니다.

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

CSV 파일을 간단히 살펴보세요.

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## 파이프라인 생성하기

여기서의 파이프라인은 [Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)에서 생성한 파이프라인과 매우 유사할 것입니다. 이 파이프라인은 CsvExampleGen, Trainer 및 Pusher의 세 가지로 구성됩니다. 그러나 우리는 특별한 Trainer 및 Pusher 구성요소를 사용할 것입니다. Trainer 구성요소는 훈련 워크로드를 Vertex AI로 이동하고 Pusher 구성요소는 훈련된 ML 모델을 파일 시스템 대신 Vertex AI에 게시합니다.

TFX는 Vertex AI Training 서비스에 훈련 작업을 제출할 수 있는 특별 `Trainer`를 제공합니다. 우리가 해야 할 일은 몇 가지 필수 GCP 매개변수와 함께 표준 `Trainer` 구성요소 대신 확장 모듈에서 `Trainer`를 사용하는 것입니다.

이 튜토리얼에서는 먼저 CPU만 사용하여 Vertex AI Training 작업을 실행한 다음 GPU를 사용하여 실행합니다.

TFX는 또한 <em>Vertex AI 모델</em>에 모델을 업로드하기 위한 특별한 <code>Pusher</code>를 제공합니다. `Pusher`는 *Vertex AI 엔드포인트* 리소스도 생성하여 온라인 예측을 제공합니다. Vertex AI에서 제공하는 온라인 예측에 대한 자세한 내용은 [Vertex AI 문서](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)를 참조하세요.

### 모델 코드를 작성합니다.

모델 자체는 [단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)의 모델과 거의 유사합니다.

[TensorFlow 배포 전략](https://www.tensorflow.org/guide/distributed_training)을 생성하는 `_get_distribution_strategy()` 함수를 추가할 것이며 GPU가 사용 가능한 경우 MirroredStrategy를 사용하기 위해 `run_fn`에서 이 함수가 사용됩니다.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified run_fn() to add distribution_strategy.

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
        for feature in _FEATURE_KEYS
    }, _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# NEW: Read `use_gpu` from the custom_config of the Trainer.
#      if it uses GPU, enable MirroredStrategy.
def _get_distribution_strategy(fn_args: tfx.components.FnArgs):
  if fn_args.custom_config.get('use_gpu', False):
    logging.info('Using MirroredStrategy with one GPU.')
    return tf.distribute.MirroredStrategy(devices=['device:GPU:0'])
  return None


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  # NEW: If we have a distribution strategy, build a model in a strategy scope.
  strategy = _get_distribution_strategy(fn_args)
  if strategy is None:
    model = _make_keras_model()
  else:
    with strategy.scope():
      model = _make_keras_model()

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

파이프라인 구성요소에서 액세스할 수 있는 GCS에 모듈 파일을 복사합니다.

그렇지 않으면 모듈 파일을 포함하는 컨테이너 이미지를 빌드하고 이 이미지를 사용하여 파이프라인 및 AI 플랫폼 훈련 작업을 실행할 수 있습니다.

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### 파이프라인 정의 작성하기

TFX 파이프라인을 생성하는 함수를 정의합니다. [단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)과 동일한 세 가지 구성요소가 있지만 GCP 확장 모듈에서 `Trainer` 및 `Pusher` 구성요소를 사용합니다.

`tfx.extensions.google_cloud_ai_platform.Trainer`는 일반 `Trainer`처럼 작동하지만 모델 학습을 위한 계산을 클라우드로 이동합니다. Vertex AI 훈련 서비스에서 사용자 지정 작업을 시작하고 오케스트레이션 시스템의 트레이너 구성요소는 Vertex AI 훈련 작업이 완료될 때까지 대기합니다.

`tfx.extensions.google_cloud_ai_platform.Pusher`는 학습된 모델을 사용하여 Vertex AI 모델과 Vertex AI 엔드포인트를 생성합니다.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, endpoint_name: str, project_id: str,
                     region: str, use_gpu: bool) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Configuration for Vertex AI Training.
  # This dictionary will be passed as `CustomJobSpec`.
  vertex_job_spec = {
      'project': project_id,
      'worker_pool_specs': [{
          'machine_spec': {
              'machine_type': 'n1-standard-4',
          },
          'replica_count': 1,
          'container_spec': {
              'image_uri': 'gcr.io/tfx-oss-public/tfx:{}'.format(tfx.__version__),
          },
      }],
  }
  if use_gpu:
    # See https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype
    # for available machine types.
    vertex_job_spec['worker_pool_specs'][0]['machine_spec'].update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })

  # Trains a model using Vertex AI Training.
  # NEW: We need to specify a Trainer for GCP with related configs.
  trainer = tfx.extensions.google_cloud_ai_platform.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5),
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.TRAINING_ARGS_KEY:
              vertex_job_spec,
          'use_gpu':
              use_gpu,
      })

  # NEW: Configuration for pusher.
  vertex_serving_spec = {
      'project_id': project_id,
      'endpoint_name': endpoint_name,
      # Remaining argument is passed to aiplatform.Model.deploy()
      # See https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#deploy_the_model
      # for the detail.
      #
      # Machine type is the compute resource to serve prediction requests.
      # See https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#machine-types
      # for available machine types and acccerators.
      'machine_type': 'n1-standard-4',
  }

  # Vertex AI provides pre-built containers with various configurations for
  # serving.
  # See https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
  # for available container images.
  serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest'
  if use_gpu:
    vertex_serving_spec.update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })
    serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-6:latest'

  # NEW: Pushes the model to Vertex AI.
  pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Vertex 파이프라인에서 파이프라인을 실행합니다

[Vertex 파이프라인용 단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple)에서 했던 것처럼 Vertex 파이프라인을 사용하여 파이프라인을 실행합니다.

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # We will use CPUs only for now.
        use_gpu=False))

생성된 정의 파일은 `google-cloud-aiplatform` 패키지의 Google Cloud aiplatform 클라이언트를 사용하여 제출할 수 있습니다.

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

이제 위의 출력에 있는 링크를 방문하거나 [Google Cloud Console](https://console.cloud.google.com/)에서 'Vertex AI &gt; 파이프라인'을 방문하여 진행 상황을 확인할 수 있습니다.

## 예측 요청으로 테스트하기

파이프라인이 완료되면 'Vertex AI &gt; 엔드포인트'의 엔드포인트 중 하나에서 *배포된* 모델을 찾을 수 있습니다. 새 엔드포인트에 예측 요청을 보내려면 엔드포인트의 ID를 알아야 합니다. 이것은 위에서 입력한 *엔드포인트 이름*과는 다릅니다. `Google Cloud Console`의 [엔드포인트 페이지](https://console.cloud.google.com/vertex-ai/endpoints)에서 ID를 찾을 수 있습니다. 이것은 매우 긴 숫자처럼 보입니다.

**실행하기 전에 아래에 ENDPOINT_ID를 설정하세요**


In [ ]:
ENDPOINT_ID=''     # <--- ENTER THIS
if not ENDPOINT_ID:
    from absl import logging
    logging.error('Please set the endpoint id.')

동일한 aiplatform 클라이언트를 사용하여 엔드포인트에 요청을 보냅니다. 펭귄 종 분류에 대한 예측 요청을 보낼 것입니다. 입력은 우리가 사용한 네 가지 요소이며, 모델은 각 종에 대해 하나의 값을 출력하기 때문에 세 개의 값이 반환됩니다.

예를 들어, 다음 특정 예는 인덱스 '2'에서 가장 큰 값을 가지며 '2'를 인쇄합니다.


In [ ]:
# docs_infra: no_execute
import numpy as np

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'culmen_length_mm':[0.71],
    'culmen_depth_mm':[0.38],
    'flipper_length_mm':[0.98],
    'body_mass_g': [0.78],
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)

# Uses argmax to find the index of the maximum value.
print('species:', np.argmax(response.predictions[0]))

온라인 예측에 대한 자세한 내용은 `Google Cloud Console`의 [엔드포인트 페이지](https://console.cloud.google.com/vertex-ai/endpoints)를 참조하세요. 샘플 요청을 보내는 방법과 더 많은 리소스에 대한 링크를 찾을 수 있습니다.

## GPU를 사용하여 파이프라인 실행하기


Vertex AI는 GPU 지원을 포함하여 다양한 시스템 유형을 사용한 훈련을 지원합니다. 사용 가능한 옵션은 [시스템 사양 참조](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype)를 참조하세요.

GPU 훈련을 지원하기 위해 이미 파이프라인을 정의했습니다. `use_gpu` 플래그를 True로 설정하기만 하면 됩니다. 그런 다음 하나의 NVIDIA_TESLA_K80을 포함하는 시스템 사양으로 파이프라인이 생성되고 모델 훈련 코드는 `tf.distribute.MirroredStrategy`를 사용합니다.

`use_gpu` 플래그는 Vertex 또는 TFX API의 일부가 아닙니다. 이 튜토리얼에서 훈련 코드를 제어하는 데만 사용됩니다.

In [ ]:
# docs_infra: no_execute
runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # Updated: Use GPUs. We will use a NVIDIA_TESLA_K80 and 
        # the model code will use tf.distribute.MirroredStrategy.
        use_gpu=True))

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

이제 위의 출력에 있는 링크를 방문하거나 [Google Cloud Console](https://console.cloud.google.com/)에서 'Vertex AI &gt; 파이프라인'을 방문하여 진행 상황을 확인할 수 있습니다.

## 정리

이 튜토리얼에서는 Vertex AI 모델과 엔드포인트를 만들었습니다. [엔드포인트](https://console.cloud.google.com/vertex-ai/endpoints)로 이동하고 먼저 엔드포인트에서 모델의 *배포를 취소*하여 원하지 않는 비용이 발생하지 않도록 이러한 리소스를 삭제하세요. 그런 다음 엔드포인트와 모델을 개별적으로 삭제할 수 있습니다.